In [1]:
import dataloader
import tomotopy as tp
from itertools import chain
import tqdm
import pandas as pd
import numpy as np

Load new data:

In [ ]:
POSTPATH = './crowd/train/shared_task_posts.csv'
LABELPATH = './crowd/train/crowd_train.csv'
USERPATH = './crowd/train/task_C_train.posts.csv'

users = dataloader.load_user_subset_from_train(USERPATH, subset = 1000)
    
user_to_post, post_to_words, post_to_metadata = dataloader.load_posts(POSTPATH, user_subset = users)
post_to_label = dataloader.load_classification(LABELPATH, user_to_post, post_to_words, post_to_metadata)
filtered_data, sw_posts, sw_timestamps = dataloader.filter_posts(post_to_label, post_to_metadata)

filtered_data = dataloader.filter_stopwords(filtered_data)
sw_posts = dataloader.filter_stopwords(sw_posts)

FOLDERPATH = './crowd_processed/'
dataloader.save_to_folder(FOLDERPATH, user_to_post, post_to_metadata, filtered_data, sw_posts, sw_timestamps)

Load existing Data:

In [2]:
FOLDERPATH = './crowd_processed/'
user_to_post, post_to_metadata, filtered_data, sw_posts, sw_timestamps = dataloader.load_from_folder(FOLDERPATH)
filtered_data = dataloader.filter_stopwords(filtered_data)
sw_posts = dataloader.filter_stopwords(sw_posts)

In [19]:
mdl = tp.SLDAModel(k=30, vars=['b'])
for data in tqdm.tqdm(filtered_data.keys()):
    mdl.add_doc(chain.from_iterable(filtered_data[data][1]), [1 if filtered_data[data][2] == 'd' else 0])

for i in range(0, 1000, 10):
    mdl.train(10)
    if (i % 100 == 0):
        print('Iteration: {}\tLog-likelihood: {}'.format(i, mdl.ll_per_word))
print("Training Finished")

100%|████████████████████████████████████████████████████████████████████████| 55255/55255 [00:00<00:00, 124781.35it/s]


Iteration: 0	Log-likelihood: -10.155441293263952
Iteration: 100	Log-likelihood: -9.214060881060316
Iteration: 200	Log-likelihood: -9.114190279724967
Iteration: 300	Log-likelihood: -9.08253916761422
Iteration: 400	Log-likelihood: -9.073029999986009
Iteration: 500	Log-likelihood: -9.066007807153923
Iteration: 600	Log-likelihood: -9.05862781614594
Iteration: 700	Log-likelihood: -9.051351661276335
Iteration: 800	Log-likelihood: -9.046680493046948
Iteration: 900	Log-likelihood: -9.045033356610785


In [20]:
slda_coefficients = mdl.get_regression_coef(0)
data = []
for k in range(mdl.k):
    top_words = mdl.get_topic_words(k, top_n=40)
    words = [word for (word, float) in top_words]
    words = ", ".join(words)
    data.append([words, slda_coefficients[k]])
    
indices = np.array(slda_coefficients).argsort()
data = np.array(data)
data = data[indices]

pd.DataFrame(data, columns=["Topic", "Suicidality Coefficient"])

,Topic,Suicidality Coefficient
0,"͡°, bike, url, ͜ʖ, person, city, area, ride, r...",-5.436562538146973
1,"buy, price, bike, sell, car, new, money, bough...",-4.356998920440674
2,"person, new, retail, sold, used, 300, pokemon,...",-4.113407135009766
3,"url, person, gun, range, button, model, press,...",-3.3555142879486084
4,"person, iv, weed, smoke, smoking, one, 0, 5, 4...",-2.645719051361084
5,"person, x, 2, humble, 2url, bundle, editionurl...",-2.633164405822754
6,"person, oxycodone, email, mg, cheap, personper...",-2.236701488494873
7,"game, play, games, playing, person, played, te...",-2.147989273071289
8,"music, person, song, english, language, sound,...",-2.1095125675201416
9,"would, one, think, people, way, could, time, m...",-2.036754846572876


In [25]:
print(data[27][0])

im, dont, like, know, get, want, really, feel, cant, people, even, think, time, make, going, go, ive, something, right, one, much, need, say, anything, way, someone, things, thats, life, see, always, youre, doesnt, good, thing, person, still, every, tell, help


In [26]:
print(data[28][0])

person, pain, doctor, hospital, mental, health, medication, anxiety, appointment, depression, symptoms, disorder, take, blood, meds, went, issues, bipolar, medical, also, ich, birth, doctors, im, treatment, pills, help, taking, normal, drug, und, er, drugs, severe, suicide, die, due, surgery, illness, patients


In [27]:
print(data[29][0])

friends, like, feel, life, never, love, want, relationship, talk, girl, people, friend, years, really, things, sex, im, would, know, always, school, dont, ive, year, women, ever, felt, much, time, told, shes, think, someone, wanted, social, together, parents, girlfriend, depression, boyfriend
